In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TODO: prendi tag, tieni solo le righe con tag umani/famosi, rimuovi duplicati, tieni solo le righe di cui abbiamo i testi. guarda cosa ci rimane.

In [2]:
#!pip install gdown -q
!pip install --upgrade --no-cache-dir -q gdown

In [15]:
import pandas as pd
import numpy as np

import os, json
import urllib.request
from tqdm import tqdm
import shutil

import sqlite3
import gdown

!rm -r sample_data

rm: cannot remove 'sample_data': No such file or directory


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download the files and create the songs dataset

In the following cells you can choose between downloading and creating the dataset from the [Million Song Dataset](http://millionsongdataset.com/musixmatch/) (the Musixmatch version) or copying the already created files from Drive.

In [ ]:
#@title #### Download and create the datasets
#url_script = ""
#gdown.download(id=url_script, output="create_dataset.py", quiet=True)

!python create_dataset.py

In [ ]:
#@title #### Load the files from Drive
import gdown

# File ids in Drive
url_song_data = "1-1qWoNBQhIerh74mo4NbMzUCJqgpDXkx"
url_eval_data = "1U8mNVgoC2kwzTwalc0oAJEEIHXOIyL4N"

gdown.download(id=url_song_data, output="songs_data.csv", quiet=False)
gdown.download(id=url_eval_data, output="eval_similar_songs.csv", quiet=False)

In [18]:
songs_df = pd.read_csv('songs_data.csv')
eval_df = pd.read_csv('eval_similar_songs.csv')